# Fine-tune ESM with ProteinGym

In [1]:
import pandas as pd
from app.helpers.sequence_util import get_loci_set

In [2]:
import pandas as pd
from io import StringIO

evolvepro_dms_str = '''Protein,Reference,Organism,Fitness Setting,Cutoff,Population Successes,Population Size
ADRB2,Jones et al., 2019,Human,Signal transduction + pathway reporter,>2.8,914,7800
β-lactamase,Stiffler et al., 2015,Bacteria,"Antibiotic resistance (ampicillin, 2500 µg/mL)",>0.01,393,4978
Env,Haddox et al., 2016,Virus,Viral replication fitness,>0.1,748,12863
HA H1,Doud and Bloom, 2016,Virus,Viral replication fitness,>0.1,645,10716
HA H3,Lee et al., 2018,Virus,Viral replication fitness,>0.1,714,10754
infA,Kelsic et al., 2016,Bacteria,Competitive growth,>0.98,305,1368
MAPK1,Brenan et al., 2016,Human,Competitive growth (SCH772984),>2.5,77,6810
P53,Giacomelli et al, 2018,Human,Competitive growth (etoposide),>1,905,7448
PafA,Markin et al., 2021,Bacteria,"Kcat/KM P < 0.01, faster than WT",-,35,1040
AsCas12f,Hino et al., 2023,Bacteria,Genomic DNA cleavage,>1,1436,7941
Zika Envelope Protein,Sourisseau et al., 2019,Virus,Viral replication fitness,>1,351,9576
COV2 Spike Protein,Greaney et al., 2021,Virus,Yeast display binding,>0.05,232,1959'''

# Read the CSV properly, handling commas inside quoted fields
evolvepro_dms_df = pd.read_csv(StringIO(evolvepro_dms_str), quotechar='"', index_col=False)
evolvepro_dms_df['author'] = evolvepro_dms_df.Reference.apply(lambda v: v.split(' ')[0].strip().lower())

# Display the DataFrame
evolvepro_dms_df

/tmp/ipykernel_192/4128782694.py:19: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  evolvepro_dms_df = pd.read_csv(StringIO(evolvepro_dms_str), quotechar='"', index_col=False)


,Protein,Reference,Organism,Fitness Setting,Cutoff,Population Successes,Population Size,author
0,ADRB2,Jones et al.,2019,Human,Signal transduction + pathway reporter,>2.8,914,jones
1,β-lactamase,Stiffler et al.,2015,Bacteria,"Antibiotic resistance (ampicillin, 2500 µg/mL)",>0.01,393,stiffler
2,Env,Haddox et al.,2016,Virus,Viral replication fitness,>0.1,748,haddox
3,HA H1,Doud and Bloom,2016,Virus,Viral replication fitness,>0.1,645,doud
4,HA H3,Lee et al.,2018,Virus,Viral replication fitness,>0.1,714,lee
5,infA,Kelsic et al.,2016,Bacteria,Competitive growth,>0.98,305,kelsic
6,MAPK1,Brenan et al.,2016,Human,Competitive growth (SCH772984),>2.5,77,brenan
7,P53,Giacomelli et al,2018,Human,Competitive growth (etoposide),>1,905,giacomelli
8,PafA,Markin et al.,2021,Bacteria,"Kcat/KM P < 0.01, faster than WT",-,35,markin
9,AsCas12f,Hino et al.,2023,Bacteria,Genomic DNA cleavage,>1,1436,hino


In [3]:
pg_df = pd.read_csv('notebooks/data/DMS_substitutions.csv')
pg_df['author'] = pg_df.DMS_id.apply(lambda v: v.split('_')[2].lower())
pg_df.head(5)

,DMS_id,DMS_filename,UniProt_ID,taxon,source_organism,target_seq,seq_len,includes_multiple_mutants,DMS_total_number_mutants,DMS_number_single_mutants,...,raw_DMS_phenotype_name,raw_DMS_directionality,raw_DMS_mutant_column,weight_file_name,pdb_file,pdb_range,ProteinGym_version,raw_mut_offset,coarse_selection_type,author
0,A0A140D2T1_ZIKV_Sourisseau_2019,A0A140D2T1_ZIKV_Sourisseau_2019.csv,A0A140D2T1_ZIKV,Virus,Zika virus (ZIKV),MKNPKKKSGGFRIVNMLKRGVARVNPLGGLKRLPAGLLLGHGPIRM...,3423,False,9576,9576,...,effect,1,mutant,A0A140D2T1_ZIKV_theta_0.01.npy,A0A140D2T1_ZIKV.pdb,291-794,0.1,NaN,OrganismalFitness,sourisseau
1,A0A192B1T2_9HIV1_Haddox_2018,A0A192B1T2_9HIV1_Haddox_2018.csv,A0A192B1T2_9HIV1,Virus,Human immunodeficiency virus 1,MRVKGIQMNSQHLLRWGIMILGMIMICSVAGNLWVTVYYGVPVWKD...,852,False,12577,12577,...,fitness,1,mutant,A0A192B1T2_9HIV1_theta_0.01.npy,A0A192B1T2_9HIV1.pdb,1-852,0.1,NaN,OrganismalFitness,haddox
2,A0A1I9GEU1_NEIME_Kennouche_2019,A0A1I9GEU1_NEIME_Kennouche_2019.csv,A0A1I9GEU1_NEIME,Prokaryote,Neisseria meningitidis,FTLIELMIVIAIVGILAAVALPAYQDYTARAQVSEAILLAEGQKSA...,161,False,922,922,...,piliation_log2_ratio,1,mutants,A0A1I9GEU1_NEIME_theta_0.2.npy,A0A1I9GEU1_NEIME.pdb,1-161,0.1,NaN,Activity,kennouche
3,A0A247D711_LISMN_Stadelmann_2021,A0A247D711_LISMN_Stadelmann_2021.csv,A0A247D711_LISMN,Prokaryote,Listeria monocytogenes,MNINDLIREIKNKDYTVKLSGTDSNSITQLIIRVNNDGNEYVISES...,87,False,1653,1653,...,mean_prediction,1,mutant,A0A247D711_LISMN_b03_theta_0.2.npy,A0A247D711_LISMN.pdb,1-87,1.0,NaN,Activity,stadelmann
4,A0A2Z5U3Z0_9INFA_Doud_2016,A0A2Z5U3Z0_9INFA_Doud_2016.csv,A0A2Z5U3Z0_9INFA,Virus,Influenza A virus (A/WSN/1933(H1N1)),MKAKLLVLLYAFVATDADTICIGYHANNSTDTVDTILEKNVAVTHS...,565,False,10715,10715,...,transformed_pref,1,mutant,A0A2Z5U3Z0_9INFA_theta_0.01.npy,A0A2Z5U3Z0_9INFA.pdb,1-565,0.1,NaN,OrganismalFitness,doud


In [5]:
merged_df = pd.merge(
  evolvepro_dms_df,
  pg_df,
  left_on='author',
  right_on='author',
  how='left'
)
merged_df = merged_df[
  (merged_df.author != 'haddox') | 
  (merged_df.DMS_id == 'ENV_HV1BR_Haddox_2016')
]
merged_df = merged_df[
  (merged_df.author != 'doud') | 
  (merged_df.DMS_id == 'A0A2Z5U3Z0_9INFA_Doud_2016')
]
merged_df = merged_df[
  (merged_df.author != 'giacomelli') | 
  (merged_df.DMS_id == 'P53_HUMAN_Giacomelli_2018_Null_Etoposide')
]
merged_df = merged_df[~merged_df.DMS_id.isna()]
merged_df = merged_df.reset_index(drop=True)
merged_df

,Protein,Reference,Organism,Fitness Setting,Cutoff,Population Successes,Population Size,author,DMS_id,DMS_filename,...,raw_DMS_filename,raw_DMS_phenotype_name,raw_DMS_directionality,raw_DMS_mutant_column,weight_file_name,pdb_file,pdb_range,ProteinGym_version,raw_mut_offset,coarse_selection_type
0,ADRB2,Jones et al.,2019,Human,Signal transduction + pathway reporter,>2.8,914,jones,ADRB2_HUMAN_Jones_2020,ADRB2_HUMAN_Jones_2020.csv,...,ADRB2_HUMAN_Jones_2020.csv,0.625,1.0,mutant_id,ADRB2_HUMAN_theta_0.2.npy,ADRB2_HUMAN.pdb,1-413,0.1,NaN,Activity
1,β-lactamase,Stiffler et al.,2015,Bacteria,"Antibiotic resistance (ampicillin, 2500 µg/mL)",>0.01,393,stiffler,BLAT_ECOLX_Stiffler_2015,BLAT_ECOLX_Stiffler_2015.csv,...,BLAT_ECOLX_Stiffler_2015.csv,2500,1.0,mutant,BLAT_ECOLX_theta_0.2.npy,BLAT_ECOLX.pdb,1-286,0.1,NaN,OrganismalFitness
2,Env,Haddox et al.,2016,Virus,Viral replication fitness,>0.1,748,haddox,ENV_HV1BR_Haddox_2016,ENV_HV1BR_Haddox_2016.csv,...,ENV_HV1BR_Haddox_2016.csv,score,1.0,mutant,ENV_HV1BR_theta_0.01.npy,ENV_HV1BR.pdb,1-861,0.1,NaN,OrganismalFitness
3,HA H1,Doud and Bloom,2016,Virus,Viral replication fitness,>0.1,645,doud,A0A2Z5U3Z0_9INFA_Doud_2016,A0A2Z5U3Z0_9INFA_Doud_2016.csv,...,A0A2Z5U3Z0_9INFA_Doud_2016.csv,transformed_pref,1.0,mutant,A0A2Z5U3Z0_9INFA_theta_0.01.npy,A0A2Z5U3Z0_9INFA.pdb,1-565,0.1,NaN,OrganismalFitness
4,HA H3,Lee et al.,2018,Virus,Viral replication fitness,>0.1,714,lee,C6KNH7_9INFA_Lee_2018,C6KNH7_9INFA_Lee_2018.csv,...,C6KNH7_9INFA_Lee_2018.csv,log_fitness_by_syn_mut_fitness,1.0,mutant,C6KNH7_9INFA_theta_0.01.npy,C6KNH7_9INFA.pdb,1-566,0.1,NaN,OrganismalFitness
5,infA,Kelsic et al.,2016,Bacteria,Competitive growth,>0.98,305,kelsic,IF1_ECOLI_Kelsic_2016,IF1_ECOLI_Kelsic_2016.csv,...,IF1_ECOLI_Kelsic_2016.csv,fitness_rich,1.0,mutant,IF1_ECOLI_theta_0.2.npy,IF1_ECOLI.pdb,1-72,0.1,NaN,OrganismalFitness
6,MAPK1,Brenan et al.,2016,Human,Competitive growth (SCH772984),>2.5,77,brenan,MK01_HUMAN_Brenan_2016,MK01_HUMAN_Brenan_2016.csv,...,MK01_HUMAN_Brenan_2016.csv,DOX_Average,-1.0,mutant,MK01_HUMAN_theta_0.2.npy,MK01_HUMAN.pdb,1-360,0.1,NaN,OrganismalFitness
7,P53,Giacomelli et al,2018,Human,Competitive growth (etoposide),>1,905,giacomelli,P53_HUMAN_Giacomelli_2018_Null_Etoposide,P53_HUMAN_Giacomelli_2018_Null_Etoposide.csv,...,P53_HUMAN_Giacomelli_2018.csv,A549_p53NULL_Etoposide_Z-score,1.0,Allele,P53_HUMAN_theta_0.2.npy,P53_HUMAN.pdb,1-393,0.1,NaN,OrganismalFitness
8,Zika Envelope Protein,Sourisseau et al.,2019,Virus,Viral replication fitness,>1,351,sourisseau,A0A140D2T1_ZIKV_Sourisseau_2019,A0A140D2T1_ZIKV_Sourisseau_2019.csv,...,A0A140D2T1_ZIKV_Sourisseau_growth_2019.csv,effect,1.0,mutant,A0A140D2T1_ZIKV_theta_0.01.npy,A0A140D2T1_ZIKV.pdb,291-794,0.1,NaN,OrganismalFitness


In [6]:
merged_df.selection_assay.value_counts()

selection_assay
Viral replication                                                               3
Growth                                                                          2
transcription (luciferase reporter, isoproterenol (beta2AR agonist)-induced)    1
Growth (10-2500 ug/mL ampicillin)                                               1
viral replication                                                               1
drug resistance (nutlin-3, etoposide)                                           1
Name: count, dtype: int64

In [54]:
good_pg_dfs = pg_df[
  pg_df.selection_assay.apply(lambda v: (
    ('stability' not in str(v).lower()) and
    ('abundance' not in str(v).lower()) and
    ('surrogate for toxicity' not in str(v).lower())
  ))
  & (~pg_df.includes_multiple_mutants)
  & (~pg_df.DMS_id.isin(merged_df.DMS_id))
  & (~pg_df.UniProt_ID.isin(merged_df.UniProt_ID))
  & (~pg_df.author.isin(merged_df.author))
  & (pg_df.taxon != 'Virus')  # We don't train on viruses.
  & (pg_df.DMS_number_single_mutants > 1000)
  & (pg_df.seq_len < 1024)
].sort_values(by='DMS_number_single_mutants', ascending=False)

assert len(set(good_pg_dfs.UniProt_ID) & set(merged_df.UniProt_ID)) == 0, 'Found UniProt IDs in both training and validation sets.'
assert len(set(good_pg_dfs.pdb_file) & set(merged_df.pdb_file)) == 0, 'Found pdb files in both training and validation sets.'
assert len(set(good_pg_dfs.author) & set(merged_df.author)) == 0, 'Found authors in both training and validation sets.'

good_pg_dfs.shape

(66, 46)

In [48]:
good_pg_dfs.head(10).columns

Index(['DMS_id', 'DMS_filename', 'UniProt_ID', 'taxon', 'source_organism',
       'target_seq', 'seq_len', 'includes_multiple_mutants',
       'DMS_total_number_mutants', 'DMS_number_single_mutants',
       'DMS_number_multiple_mutants', 'DMS_binarization_cutoff',
       'DMS_binarization_method', 'first_author', 'title', 'year', 'jo',
       'region_mutated', 'molecule_name', 'selection_assay', 'selection_type',
       'MSA_filename', 'MSA_start', 'MSA_end', 'MSA_len', 'MSA_bitscore',
       'MSA_theta', 'MSA_num_seqs', 'MSA_perc_cov', 'MSA_num_cov', 'MSA_N_eff',
       'MSA_Neff_L', 'MSA_Neff_L_category', 'MSA_num_significant',
       'MSA_num_significant_L', 'raw_DMS_filename', 'raw_DMS_phenotype_name',
       'raw_DMS_directionality', 'raw_DMS_mutant_column', 'weight_file_name',
       'pdb_file', 'pdb_range', 'ProteinGym_version', 'raw_mut_offset',
       'coarse_selection_type', 'author'],
      dtype='object')

In [55]:
import numpy as np
# set numpy random key.
np.random.seed(42)


def get_training_pairs_for_dms(dms_id, dms_fname, dms_target_seq, num_examples):
  dms_df = pd.read_csv(f'notebooks/data/DMS_ProteinGym_substitutions/{dms_fname}')
  dms_df = dms_df.sort_values('DMS_score', ascending=False).reset_index(drop=True)

  training_pairs = []
  for ii in range(num_examples):
    # Randomly select one of the top 10% mutants.
    # Then randomly select another mutant at least 10% lower on the list.
    example_w_idx = np.random.randint(0, int(len(dms_df) * 0.1))
    example_l_idx = np.random.randint(example_w_idx + int(len(dms_df) * 0.1), len(dms_df))
    example_w = dms_df.iloc[example_w_idx]
    example_l = dms_df.iloc[example_l_idx]

    # if get_loci_set(example_w.mutant) == get_loci_set(example_l.mutant):
    #   print(f'Skipping a duplicate locus set {example_w.mutant} and {example_l.mutant}')
    #   continue

    training_pairs.append(pd.DataFrame({
      'dms_id': [dms_id],
      'sequence': [dms_target_seq],
      'seq_id_w': [example_w.mutant],
      'seq_id_l': [example_l.mutant],
      'rank_w': [example_w_idx],
      'rank_l': [example_l_idx],
      'DMS_score_w': [example_w.DMS_score],
      'DMS_score_l': [example_l.DMS_score],
    }))
  return pd.concat(training_pairs)

def get_full_training_pairs_df(dataset_df, num_examples_per_dms):
  training_pairs = []
  for row in dataset_df.itertuples():
    training_pairs.append(get_training_pairs_for_dms(row.DMS_id, row.DMS_filename, row.target_seq, num_examples_per_dms))
  return pd.concat(training_pairs, ignore_index=True)

def build_train_and_valid_df(num_examples_train, num_examples_valid):
  train_df = get_full_training_pairs_df(good_pg_dfs, num_examples_train).assign(use_for_validation=False)
  valid_df = get_full_training_pairs_df(merged_df, num_examples_valid).assign(use_for_validation=True)
  return pd.concat([train_df, valid_df], ignore_index=True)

In [64]:
build_train_and_valid_df(1, 1).to_excel('notebooks/data/training_pairs/pairs1.xlsx')
build_train_and_valid_df(10, 1).to_excel('notebooks/data/training_pairs/pairs10.xlsx')
build_train_and_valid_df(100, 10).to_excel('notebooks/data/training_pairs/pairs100.xlsx')
build_train_and_valid_df(1000, 100).to_excel('notebooks/data/training_pairs/pairs1000.xlsx')
build_train_and_valid_df(10000, 250).to_excel('notebooks/data/training_pairs/pairs10000.xlsx')

In [60]:
merged_df.UniProt_ID

0         ADRB2_HUMAN
1          BLAT_ECOLX
2           ENV_HV1BR
3    A0A2Z5U3Z0_9INFA
4        C6KNH7_9INFA
5           IF1_ECOLI
6          MK01_HUMAN
7           P53_HUMAN
8     A0A140D2T1_ZIKV
Name: UniProt_ID, dtype: object

In [56]:
good_pg_dfs[good_pg_dfs.DMS_id == 'OPSD_HUMAN_Wan_2019']

,DMS_id,DMS_filename,UniProt_ID,taxon,source_organism,target_seq,seq_len,includes_multiple_mutants,DMS_total_number_mutants,DMS_number_single_mutants,...,raw_DMS_phenotype_name,raw_DMS_directionality,raw_DMS_mutant_column,weight_file_name,pdb_file,pdb_range,ProteinGym_version,raw_mut_offset,coarse_selection_type,author
111,OPSD_HUMAN_Wan_2019,OPSD_HUMAN_Wan_2019.csv,OPSD_HUMAN,Human,Homo sapiens,MNGTEGPNFYVPFSNATGVVRSPFEYPQYYLAEPWQFSMLAAYMFL...,348,False,165,165,...,score,1,mutant,OPSD_HUMAN_theta0.2_2023-10-12_b04.npy,OPSD_HUMAN.pdb,1-348,1.0,NaN,Expression,wan
